#**Financial Model**

**Calculating predictions based on the best model i.e. XGB Classifier**

In [104]:
y_fn = df['Choice']
X_fn = df.drop(columns=['Choice','dAB_closeness','dAB_following','dAB_betweenness'])
X_train, X_test, y_train, y_test = train_test_split(X_fn, y_fn, test_size=0.33, random_state=5)

from xgboost.sklearn import XGBClassifier
xgb=XGBClassifier(random_state=5, learning_rate=0.04)
model_xgb = xgb.fit(X_train, y_train)
y_test_pred_xgb = model_xgb.predict(X_test)

**Database to calculate the financial impact on the test dataset**

In [136]:
financial_df_cal = X_test.copy()

In [137]:
financial_df_cal['Influencer'] = np.where(y_test==1,'A','B')
financial_df_cal['Influencer_Pred'] = np.where(y_test_pred_xgb==1,'A','B')

In [138]:
financial_df_cal = financial_df_cal[['A_follower_count','B_follower_count','Influencer','Influencer_Pred']]

In [139]:
financial_df_cal['Correct_Decision'] = (financial_df_cal['Influencer'] == financial_df_cal['Influencer_Pred'])
financial_df_cal

,A_follower_count,B_follower_count,Influencer,Influencer_Pred,Correct_Decision
871,454514,47006,A,A,True
3385,723800,750,B,A,False
336,598,224146,B,B,True
2919,5941,263614,B,B,True
3030,871652,1043,B,A,False
...,...,...,...,...,...
2519,119001,541611,B,B,True
3412,2326,3113,B,A,False
4939,10422587,657021,A,A,True
1552,3893,592,A,A,True


**Financial Impact (With and without Analytical model)**

Information given:

In [140]:
profit_margin = 10
prob_1_tweet = 0.01/100
prob_2_tweet = 0.015/100
cost_1_tweet = 5
cost_2_tweet = 10

**Without analytical model**

In [158]:
financial_df_cal['Without_Model'] = np.where(financial_df_cal['Influencer'] == 'A',
                                             financial_df_cal['A_follower_count']*profit_margin*prob_1_tweet,
                                             financial_df_cal['B_follower_count']*profit_margin*prob_1_tweet)

In [180]:
Total_Profit_without_analytics = (financial_df_cal['Without_Model'].sum()- 2*cost_1_tweet*financial_df_cal['A_follower_count'].count())
Total_Profit_without_analytics

1667130.53

The total profit without using analytical model is **$1667131**

**With analytical model**

In [166]:
financial_df_cal['With_Model'] = np.where(financial_df_cal['Influencer_Pred'] == 'A',
                                             financial_df_cal['A_follower_count']*profit_margin*prob_2_tweet,
                                             financial_df_cal['B_follower_count']*profit_margin*prob_2_tweet)

In [178]:
Total_Profit_with_analytics = (financial_df_cal['With_Model'].sum()- (cost_2_tweet*financial_df_cal['A_follower_count'].count()))
Total_Profit_with_analytics

2902674.154

The total profit with analytical model is **$2902674**

**Model Impact**

In [182]:
Model_Impact = Total_Profit_with_analytics - Total_Profit_without_analytics
Model_Impact

1235543.624

In [184]:
(Total_Profit_with_analytics - Total_Profit_without_analytics)/Total_Profit_without_analytics

0.7411199073896152

The financial impact in the test dataset is equal to **$ 1235544**